<a href="https://colab.research.google.com/github/AlexTako/learning/blob/main/analyse%20with%20Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Используя датасет о стоимости акций Сбербанка с 01.01.2013 года:

https://storage.yandexcloud.net/academy.ai/SBER.csv

визуализируйте индикатор "Полосы Боллинджера", проанализируйте график, и предложите вариант торговли акциями Сбербанка с помощью этого инструмента.

**Подсказка.**
Индикатор выглядит как полоса из трех линий:

* линия посередине — это простая скользящая средняя (SMA) с периодом `ma_size`, обычно около 20 дней;

* верхняя и нижняя линии (BB) — построены на основе SMА, но к нему добавленно стреднеквадратичное отклонение:

```
 SMA = data['close'].rolling(ma_size).mean()
 BB_UP = SMA + data['close'].rolling(ma_size).std() * bol_size
 BB_DOWN = SMA - data['close'].rolling(ma_size).std() * bol_size
   
```

где bol_size - ширина коридора, подбирается по графику. Выберите такое его значение, чтобы по графику можно было принимать торговые решения.

In [ ]:
import os
import numpy as np
import pandas as pd

import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [ ]:
!wget https://storage.yandexcloud.net/academy.ai/SBER.csv

--2024-01-24 18:49:37--  https://storage.yandexcloud.net/academy.ai/SBER.csv
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180264 (176K) [text/csv]
Saving to: ‘SBER.csv’

SBER.csv            100%[===================>] 176.04K   398KB/s    in 0.4s    

2024-01-24 18:49:39 (398 KB/s) - ‘SBER.csv’ saved [180264/180264]



In [ ]:
df = pd.read_csv("./SBER.csv", sep = ';')
# Дата должна быть в формате datetime для построения временных рядов
df['DATE'] = pd.to_datetime(df['DATE'], format = '%Y%m%d',)

In [ ]:
#расчет индикатора полосы Боллинджера
ma_size = 20 #период (дней)
bol_size = 2.5 #ширина коридора
#SMA - расчет скользящих средних с периодом в 20 дней
df['SMA_20'] = df['CLOSE'].rolling(20).mean().shift()
df['SMA_40'] = df['CLOSE'].rolling(40).mean().shift()
df['SMA_60'] = df['CLOSE'].rolling(60).mean().shift()
df['SMA_80'] = df['CLOSE'].rolling(80).mean().shift()

SMA = df['CLOSE'].rolling(ma_size).mean() #средняя линия
BB_UP = SMA + df['CLOSE'].rolling(ma_size).std() * bol_size #верхняя линия
BB_DOWN = SMA - df['CLOSE'].rolling(ma_size).std() * bol_size #нижняя линия

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=df['DATE'], #формируем свечной график акций сбера
                open=df['OPEN'],
                high=df['HIGH'],
                low=df['LOW'],
                close=df['CLOSE'])])
fig.add_scatter(x=df['DATE'], y=SMA, name='SMA') #средняя линия
fig.add_scatter(x=df['DATE'], y=BB_UP, name='Up line')
fig.add_scatter(x=df['DATE'], y=BB_DOWN, name='Down line')
fig.update_xaxes(rangeslider_visible=True) # Обновляем ось Х, добавляя в нее ползунок диапазона
fig.show()

Исходя из анализа, мы видим коридор за пределы которого цены не выходят, а также среднюю линию.

Способ использования графика:
1.   Анализируем цену
2.   Если она приближается к верхней линии, значит цена вскоре начнет падать
3.   Когда цена достигла верхней линии и две свечи отмечают падение цены, входим в сделку sell
4.   При приближении цены к средней линии, закрываем сделку
5.   Ожидаем дальнейшего поведения графика
6.   В случае, если две свечи после преодоления средней линии отмечают рост цены, входим в сделку buy
7.   Аналогично торгуем внутри коридора


